### Imports

In [2]:
%load_ext autoreload
%autoreload 2

from openaq_anomaly_prediction.config import Configuration as config
from openaq_anomaly_prediction.utils.logging import logger, ProgressLogger
from openaq_anomaly_prediction.load.openaq import client as openaq, AreaDownloader

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from pprint import pprint

pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)
pd.set_option("display.width", 1000)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


### Get Area Data (New Delhi)

In [3]:
new_delhi = AreaDownloader(area_name="new_delhi")
new_delhi.load_bbox(76.772461, 28.161110, 77.768372, 28.943516)

logger.info(f"Area: {new_delhi.area_name.upper()}")
logger.info(f"Locations: {new_delhi.locations.shape}")
logger.info(f"Sensors: {new_delhi.sensors.shape}")

01:05:54     INFO |  Area: NEW_DELHI
01:05:54     INFO |  Locations: (107, 27)
01:05:54     INFO |  Sensors: (1241, 7)


### Create STG file (CSV)

In [ ]:
import glob
from openaq_anomaly_prediction.utils.helpers import concat_csv_to_csv
import os
# CONCATENATE: Concatenate several trimesters CSVs into a single CSV file

staging_csv_prefix = "new_delhi_2024"

selected_files = glob.glob(os.path.join(config.DATA_CSV_PATH, f"{staging_csv_prefix}*.raw.csv"))
pprint(selected_files)

['/home/deniscck/code/denis-cck/openaq_anomaly_prediction/data/csv/new_delhi_2024_2024-01-01_2024-03-31.csv',
 '/home/deniscck/code/denis-cck/openaq_anomaly_prediction/data/csv/new_delhi_2024_2024-07-01_2024-09-30.csv',
 '/home/deniscck/code/denis-cck/openaq_anomaly_prediction/data/csv/new_delhi_2024_2024-04-01_2024-06-30.csv',
 '/home/deniscck/code/denis-cck/openaq_anomaly_prediction/data/csv/new_delhi_2024_2024-10-01_2024-12-31.csv']


In [ ]:
# concat_csv_to_csv(selected_files, f"{staging_csv_prefix}.stg.csv")

01:06:21      4/4 |   󰘍 100%: Appending CSV files to final CSV -> data/csv/new_delhi_2024.stg.csv 


### Clean DataFrames

In [6]:
def get_clean_measurements(df: pd.DataFrame, area: AreaDownloader) -> pd.DataFrame:

    # ---------------------------------------------------------------------
    # CLEAN MEASUREMENTS DATAFRAME
    # print(f"{'-' * 44}\nCLEANING [MEASUREMENTS] DATAFRAME:")
    clean_measurements = df[
        [
            "sensor_id",
            "value",
            "parameter.id",
            "parameter.name",
            "parameter.units",
            "period.datetimeFrom.local",
            "period.datetimeTo.local",
            "period.datetimeFrom.utc",
            "period.datetimeTo.utc",
            "coverage.expectedCount",
            "coverage.observedCount",
        ]
    ]
    # display(clean_measurements.head(1))

    # CLEAN SENSORS DATAFRAME
    # print(f"\n{'-' * 44}\nCLEANING [SENSORS] DATAFRAME:")
    clean_sensors = new_delhi.sensors[
        [
            "location_id",
            "id",
            "name",
            "parameter.displayName",
        ]
    ]
    # display(clean_sensors.head(1))

    # CLEAN LOCATIONS DATAFRAME
    # print(f"\n{'-' * 44}\nCLEANING [LOCATIONS] DATAFRAME:")
    clean_locations = new_delhi.locations[
        [
            "id",
            "name",
            "isMobile",
            "isMonitor",  # Maybe it's whether it's recognized as an "official" monitoring station or not?
            "country.id",
            "country.code",
            "country.name",
            "owner.id",
            "owner.name",
            "provider.id",
            "provider.name",
            "coordinates.latitude",
            "coordinates.longitude",

            # KEEP BUT DON'T NEED FOR MEASUREMENTS
            # "datetimeFirst.local",
            # "datetimeLast.local",
            "datetimeFirst.utc",
            "datetimeLast.utc",

            # CUSTOM FIELDS
            # "sensors_flat",  # custom field added in AreaDownloader
            # "instruments_flat",  # TODO: custom field added in AreaDownloader.
            #    Not very standardized (sometimes duplicates) and no way of linking it to sensors/measurements.
            #    You can only know which instruments are used in a location, but not which instrument is used for which sensor/parameter.
            #    So for now we just keep it for reference but don't use it.

            # EMPTY IN NEW DELHI
            # "locality",  # TODO: ???: 106/107 empty in New Delhi
            # "bounds",  # TODO: ???: all locations have fixed coordinates (no bounds just a point)
            # "distance",  # TODO: ???: fully empty in New Delhi
            # "licenses",  # TODO: ???: Vast majority of locations have NaN here, but there are some. Even then is that really useful? IDK just drop it

            # DON'T KEEP
            # "instruments",
            # "sensors",
            # "timezone",  # all the same usually for a city-sized area, and doesn't really influence the measurements themselves
            # "datetimeFirst",  # NaT (not a time) for all locations in New Delhi
            # "datetimeLast",  # NaT (not a time) for all locations in New Delhi
        ]
    ].rename(
        columns={
            "id": "location_id",
            "name": "location_name",
            "datetimeFirst.utc": "location.datetimeFirst.utc",
            "datetimeLast.utc": "location.datetimeLast.utc",
        }
    )


    # ---------------------------------------------------------------------
    # JOIN THE MEASUREMENTS WITH THE SENSORS ON SENSOR_ID
    df_joined = clean_measurements.join(clean_sensors.set_index("id"), on="sensor_id", how="left")

    # JOIN THE PREVIOUS RESULT WITH THE LOCATIONS ON LOCATION_ID
    df_final = df_joined.join(clean_locations.set_index("location_id"), on="location_id", how="left")

    # REORDER COLUMNS
    ordered_columns = [
        "location_id",
        "sensor_id",
        "name",
        "value",
        "parameter.id",
        "parameter.name",
        "parameter.units",
        "parameter.displayName",
        "period.datetimeFrom.local",
        "period.datetimeTo.local",
        "period.datetimeFrom.utc",
        "period.datetimeTo.utc",
        "location.datetimeFirst.utc",
        "location.datetimeLast.utc",
        "coordinates.latitude",
        "coordinates.longitude",
        "location_name",
        "isMobile",
        "isMonitor",
        "country.id",
        "country.code",
        "country.name",
        "owner.id",
        "owner.name",
        "provider.id",
        "provider.name",
        "coverage.expectedCount",
        "coverage.observedCount",
    ]
    df_final = df_final[ordered_columns]

    # TODO: Move to a function and apply to all dataframes after loading/cleaning
    # Convert columns with "datetime" in their names to datetime types
    datetime_columns = [col for col in df_final.columns if "datetime" in col]
    for col in datetime_columns:
        df_final[col] = pd.to_datetime(df_final[col])

    # display(df_final.head(1))
    # print(len(df_final.columns))
    # print(f"Final measurements dataframe memory usage: {df_final.memory_usage(index=True, deep=True).sum() / 1024 ** 2:.2f} MB")

    return df_final

In [9]:
staging_csv_path = os.path.join(config.DATA_CSV_PATH, f"{staging_csv_prefix}.stg.csv")

df_stg = pd.read_csv(staging_csv_path)
print(f"{staging_csv_prefix}: {df_stg.shape}")

new_delhi_2024: (42107, 34)


In [10]:
# df_int = get_clean_measurements(df_stg, new_delhi)
df_int = new_delhi.get_clean_measurements(df_stg)

In [ ]:
# df_int.to_csv(os.path.join(config.DATA_CSV_PATH, f"{staging_csv_prefix}_measurements.int.csv"), index=False)

In [11]:
df_int.to_parquet(os.path.join(config.DATA_CSV_PATH, f"{staging_csv_prefix}_measurements.int.parquet"), index=False, compression='snappy')